In [ ]:
run_original = False

In [ ]:
import mne
import os
import numpy as np
recon_all_path = r"D:\REFTEP_ALL\REFTEP_reco"
recon_all_dirs = [directory for directory in os.listdir(recon_all_path) if "fatsuppressed" not in directory]
conductivities = [0.33, 0.0042,0.33]
ico=4

In [ ]:
if run_original:
    #make bem solutions for subjects
    failed_subjects = []
    for directory in recon_all_dirs:
        directory_path = os.path.join(recon_all_path,directory)
        for subject in os.listdir(directory_path):
            if subject != "fsaverage": #only analyze real subjects
                try:
                    bem_surfaces = mne.make_bem_model(subject, ico=ico, conductivity=conductivities, subjects_dir=directory_path)
                    bem_solution = mne.make_bem_solution(bem_surfaces)
                    bem_path_solution_path = f'{directory_path}/{subject}/bem/bem-solution'
                    mne.write_bem_solution(bem_path_solution_path, bem_solution, overwrite=True)
                except RuntimeError:
                    failed_subjects.append(subject)
                    print(f"error in creating bem model with {subject}")


In [ ]:
if run_original:
    for subject in failed_subjects:
        print(f"error in creating bem model with {subject}")
    np.save("D:\REFTEP_ALL/failed_subjects_headmodels",failed_subjects)

In [ ]:
#create copies of skull surfaces to modify
import mne
import os
import numpy as np
import shutil
failed_subjects = np.load("D:\REFTEP_ALL/failed_subjects_headmodels.npy")
print(failed_subjects)
if run_original:
    for directory in recon_all_dirs:
            directory_path = os.path.join(recon_all_path,directory)
            for subject in os.listdir(directory_path):
                if subject in failed_subjects:
                    bem_dir = f'{directory_path}/{subject}/bem/'
                    conv_dir = f'{directory_path}/{subject}/bem/conv/'
                    os.makedirs(conv_dir)
                    inner_skull_path = os.path.join(bem_dir,'inner_skull.surf')
                    inner_skull_orig_path = os.path.join(bem_dir,'inner_skull_orig.surf')
                    coords, faces = mne.read_surface(inner_skull_path)
                    mne.write_surface(os.path.join(conv_dir,'inner_skull.obj'),coords,faces,overwrite=True)
                    #create backup
                    shutil.copy(inner_skull_path,inner_skull_orig_path)

                    outer_skull_path = os.path.join(bem_dir,'outer_skull.surf')
                    outer_skull_orig_path = os.path.join(bem_dir,'outer_skull_orig.surf')
                    coords, faces = mne.read_surface(outer_skull_path)
                    mne.write_surface(os.path.join(conv_dir,'outer_skull.obj'),coords,faces,overwrite=True)
                    shutil.copy(outer_skull_path,outer_skull_orig_path)

#### You could edit the surfaces in blender or scale the inner skull surface like what has been done here

In [ ]:
#edit surfaces in using blender
#downscale the inner skull surface so that it is not intersecting
scales = np.arange(0.999,0.949,-0.001)
print(f"trying scales {scales}")
for directory in recon_all_dirs:
    directory_path = os.path.join(recon_all_path,directory)
    for subject in os.listdir(directory_path):
        if subject in failed_subjects:
            bem_dir = f'{directory_path}/{subject}/bem/'
            success=False
            try_ind = 0
            while not success:
                scale_now = scales[try_ind]
                #read the inner skull surface and shrink it uniformly
                inner_skull_orig_path = os.path.join(bem_dir,'inner_skull_orig.surf') #read the original inner_skull surface
                coords, faces = mne.read_surface(inner_skull_orig_path)
                coords = coords*scale_now
                mne.write_surface(os.path.join(bem_dir,'inner_skull.surf'),coords,faces,overwrite=True) #overwrite the inner_skull surface with the new one
                try:
                    bem_surfaces = mne.make_bem_model(subject, ico=ico, conductivity=conductivities, subjects_dir=directory_path)
                    bem_solution = mne.make_bem_solution(bem_surfaces)
                    bem_path_solution_path = f'{directory_path}/{subject}/bem/bem-solution'
                    mne.write_bem_solution(bem_path_solution_path, bem_solution, overwrite=True)
                    success = True
                    print(f"success in creating bem model with {subject} and scale {scales[try_ind]}")
                    #plot the modified surfaces
                    fig = mne.viz.plot_bem(subject, subjects_dir=directory_path)
                    fig.savefig(bem_dir + "watershed_bem_inner_skull_adjusted_layers_" + str(subject))
                except RuntimeError:
                    print(f"error in creating bem model with {subject} and scale {scales[try_ind]}")
                    try_ind += 1
